In [4]:
import pandas as pd
import numpy as np
from math import *

### Read in the data

In [6]:
df = pd.read_csv('new_york_hotels.csv', encoding='cp1252')

In [7]:
df.head()

ean_hotel_id                                     name  \
0        269955       Hilton Garden Inn Albany/SUNY Area   
1        113431     Courtyard by Marriott Albany Thruway   
2        108151                    Radisson Hotel Albany   
3        254756  Hilton Garden Inn Albany Medical Center   
4        198232  CrestHill Suites SUNY University Albany   

                 address1    city state_province postal_code  latitude  \
0     1389 Washington Ave  Albany             NY       12206  42.68751   
1  1455 Washington Avenue  Albany             NY       12206  42.68971   
2             205 Wolf Rd  Albany             NY       12205  42.72410   
3     62 New Scotland Ave  Albany             NY       12208  42.65157   
4  1415 Washington Avenue  Albany             NY       12206  42.68873   

   longitude  star_rating  high_rate  low_rate  
0  -73.81643          3.0   154.0272  124.0216  
1  -73.82021          3.0   179.0100  134.0000  
2  -73.79822          3.0   134.1700   84.1600  
3  -73.77638          3.0   308.2807  228.4597  
4  -73.81854          3.0   169.3900   89.3900

## Haversine definition

In [8]:
def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

## Crudest looping

In [30]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [31]:
%%timeit

# Run the haversine looping function
df['distance'] = haversine_looping(df)

445 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Iterrows Haversine

In [9]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985,\
                                      row['latitude'], row['longitude']))
df['distance'] = haversine_series

120 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Apply Haversine on rows

### Timing "apply"

In [10]:
%timeit df['distance'] =\
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

57.5 ms ± 4.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Profiling "apply"

Don't forget to run `pip install line_profiler` before trying to run `%lprun`!

Depending on your configuration, you may also need to run it to your ipython config files - for example, by editing the file ~/.ipython/profile_default/ipython_config.py to add the 'line_profiler' item to the extensions list:

```c.TerminalIPythonApp.extensions = ['line_profiler',]```

In [14]:
# Load the profiler into your Jupyter notebook
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [15]:
# Haversine applied on rows
%lprun -f haversine \
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

## Vectorized implementation of Haversine applied on Pandas series

#### Timing vectorized implementation

In [16]:
# Vectorized implementation of Haversine applied on Pandas series
%timeit df['distance'] = haversine(40.671, -73.985,\
                                   df['latitude'], df['longitude'])

1.03 ms ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Profiling vectorized implementation

In [17]:
# Vectorized implementation profile
%lprun -f haversine haversine(40.671, -73.985,\
                              df['latitude'], df['longitude'])

## Vectorized implementation of Haversine applied on NumPy arrays

#### Timing vectorized implementation

In [19]:
%%timeit
# See how long it takes to convert pandas arrays to NumPy ndarrays
np_lat = df['latitude'].values
np_lon = df['longitude'].values

4.92 µs ± 57.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985,\
                         df['latitude'].values, df['longitude'].values)

204 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Profiling vectorized implementation

In [53]:
%lprun -f haversine df['distance'] = haversine(40.671, -73.985,\
                        df['latitude'].values, df['longitude'].values)

## Cythonize that loop

#### Load the cython extension

In [20]:
%load_ext cython

#### Run unaltered Haversine through Cython

In [21]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef haversine_cy(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

#### Time it

In [22]:
%timeit df['distance'] =\
       df.apply(lambda row: haversine_cy(40.671, -73.985,\
                row['latitude'], row['longitude']), axis=1)

55.3 ms ± 457 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Redefine Haversine with data types and C libraries

In [28]:
%%cython -a
# Haversine cythonized
from libc.math cimport sin, cos, acos, asin, sqrt

cdef deg2rad_cy(float deg):
    cdef float rad = 0.01745329252*deg
    return rad
    
cpdef haversine_cy_dtyped(float lat1, float lon1, float lat2, float lon2):
    cdef: 
        float dlon
        float dlat
        float a
        float c
        float mi
    
    lat1, lon1, lat2, lon2 = map(deg2rad_cy, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    mi = 3959 * c
    return mi


#### Time it

In [29]:
%timeit df['distance'] =\
df.apply(lambda row: haversine_cy_dtyped(40.671, -73.985,\
                              row['latitude'], row['longitude']), axis=1)

33.3 ms ± 742 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
